In [1]:
import os.path as op
import numpy as np
import pandas as pd
from mne.io import read_epochs_eeglab
import scipy.io
from superlet import superlet, scale_from_period
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from burst_detection import extract_bursts
from fooof import FOOOF
import pickle

In [2]:
pipeline='NEARICA_behav'
epoch_types=['eye','task']

In [3]:
subjects = pd.read_csv('/home/bonaiuto/dcd_ross/data/participants.tsv', sep='\t')

subject_ids = subjects['participant_id']

max_freq=50
num=100
foi = np.linspace(1, max_freq, num)
scales = scale_from_period(1/foi)

beta_lims = [14, 32]
#beta_lims = [13.75, 33]
search_range = np.where((foi>=(beta_lims[0]-3)) & (foi<=(beta_lims[1]+3)))[0]

def process_cluster(subj_id, epoch_type, cluster):
    subject_data_dir = op.join('/home/bonaiuto/dcd_ross/data/derivatives', pipeline, subj_id, 'processed_data')
    fname = f'{subj_id}_{epoch_type}.set'
    fpath = op.join(subject_data_dir, fname)

    if not op.exists(fpath):
        return

    EEG = read_epochs_eeglab(fpath)
    EEG_data = EEG.get_data()
    
    ch_idx = np.where(np.in1d(EEG.info.ch_names, cluster))[0][0]
    clus_data = EEG_data[:, ch_idx, :]
    tf_array = np.zeros((clus_data.shape[0], len(foi), clus_data.shape[1]), dtype=np.single)

    def do_superlet(signal, scales, sfreq):
        spec = superlet(
            signal,
            samplerate=sfreq,
            scales=scales,
            order_max=40,
            order_min=1,
            c_1=4,
            adaptive=True,
        )
        return np.single(np.abs(spec))

    for ix in range(clus_data.shape[0]):
        tf_array[ix, :, :] = do_superlet(clus_data[ix, :], scales, EEG.info['sfreq'])

    psd = np.squeeze(np.mean(np.mean(tf_array, axis=-1), axis=0))
    ff = FOOOF()
    ff.fit(foi, psd, [foi[0], foi[-1]])
    ap_fit = 10 ** ff._ap_fit
    fooof_THR = ap_fit[search_range].reshape(-1, 1)

    bursts = extract_bursts(
        clus_data,
        tf_array[:, search_range, :],
        EEG.times,
        foi[search_range],
        beta_lims,
        fooof_THR,
        EEG.info['sfreq'],
        w_size=0.26,
        regress_evoked=False
    )

    n_bursts = len(bursts['trial'])
    bursts['cluster'] = np.array([cluster] * n_bursts)
    bursts['epoch_type'] = np.array([epoch_type] * n_bursts)
    bursts['subject'] = np.array([subj_id] * n_bursts)
    ivd = {v: k for k, v in EEG.event_id.items()}
    bursts['condition'] = np.array([ivd[key] for key in EEG.events[bursts['trial'], 2]])

    output_file = op.join(subject_data_dir, f'bursts_{subj_id}_{epoch_type}_{cluster}.pickle')
    with open(output_file, "wb") as fp:
        pickle.dump(bursts, fp)

# --- Parallel Execution ---
Parallel(n_jobs=-1)(
    delayed(process_cluster)(subj_id, epoch_type, cluster)
    for subj_id in subject_ids
    for epoch_type in epoch_types
    for cluster in ['C3', 'C4']
)

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C19/processed_data/C19_eye.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C07/processed_data/C07_eye.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C13/processed_data/C13_eye.set...
Not setting metadata
6 matching events found
Not setting metadata
Not setting metadata
5 matching events found
4 matching events found
No baseline correction applied
0 projection items activated
Ready.
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C09/processed_data/C09_eye.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C08/processed_data/C08_eye.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C19/processed_data/C19_eye.set...
No baseline corre

72 matching events found
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C19/processed_data/C19_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C19/processed_data/C19_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C03/processed_data/C03_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C10/processed_data/C10_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C13/processed_data/C13_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C03/processed_data/C03_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C05/processed_data/C05_task.set...
Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/

Not setting metadata
42 matching events found
No baseline correction applied
0 projection items activated
Ready.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C21/processed_data/C21_task.set...
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Ready.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bou


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C25/processed_data/C25_task.set...
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated
Ready.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivat


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivatives/NEARICA_behav_v3/C29/processed_data/C29_eye.set...
Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Ready.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Extracting parameters from /home/bonaiuto/dcd_ross/data/derivativ

...


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a low


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a low


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.


FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.49 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a low